Based on https://colab.research.google.com/drive/1205ubqebM0OsZa1nRgbGJBtitgHqIVv6?usp=sharing

In [ ]:
!pip install transformers==3.1.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 884.0/884.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.4/97.4 kB 7.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 13.4 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: Could not build wheels for tokenizers, which is required to install pyproject.toml-based projects


In [ ]:
!pip install spacy
!pip install jieba

In [ ]:
import torch
import transformers
import itertools

import spacy
import jieba

In [ ]:
"""Load the multilingual BERT model and its tokenizer."""

model = transformers.BertModel.from_pretrained('bert-base-multilingual-cased')
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-multilingual-cased')

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [ ]:
# Load spaCy English model
nlp_en = spacy.load("en_core_web_sm")
nlp_en.max_length = 8000000

In [ ]:
# read the scraped txt files
with open('/content/drive/MyDrive/Colab Notebooks/Big_data/Webcrawl_new_corpus/crawled_txt/cn_WHO_text.txt', 'r') as file:
    cn_DONs = file.read().replace('\n', '')
file.close()

with open('/content/drive/MyDrive/Colab Notebooks/Big_data/Webcrawl_new_corpus/crawled_txt/en_WHO_text.txt', 'r') as file:
    en_DONs = file.read().replace('\n', '')
file.close()

with open('/content/drive/MyDrive/Colab Notebooks/Big_data/Webcrawl_new_corpus/crawled_txt/cn_WHO_news_text.txt', 'r') as file:
    cn_WHO_news = file.read().replace('\n', '')
file.close()

with open('/content/drive/MyDrive/Colab Notebooks/Big_data/Webcrawl_new_corpus/crawled_txt/en_WHO_news_text.txt', 'r') as file:
    en_WHO_news = file.read().replace('\n', '')
file.close()

In [ ]:
cn_text = cn_DONs + cn_WHO_news
en_text = en_DONs + en_WHO_news

In [ ]:
len(cn_text)

2418315

In [ ]:
len(en_text)

7684031

In [ ]:
# tokenize english sentence
en_sent = [token.text for token in nlp_en(en_text)]

In [ ]:
# tokenize chinese sentence
cn_sent = list(jieba.cut(cn_text))
# and combine into a tokenized string
cn_text = " ".join(cn_sent)

In [ ]:
# pre-processing
sent_src, sent_tgt = en_text.strip().split(), cn_text.strip().split()
token_src, token_tgt = [tokenizer.tokenize(word) for word in sent_src], [tokenizer.tokenize(word) for word in sent_tgt]
wid_src, wid_tgt = [tokenizer.convert_tokens_to_ids(x) for x in token_src], [tokenizer.convert_tokens_to_ids(x) for x in token_tgt]
ids_src, ids_tgt = tokenizer.prepare_for_model(list(itertools.chain(*wid_src)), return_tensors='pt', model_max_length=tokenizer.model_max_length, truncation=True)['input_ids'], tokenizer.prepare_for_model(list(itertools.chain(*wid_tgt)), return_tensors='pt', truncation=True, model_max_length=tokenizer.model_max_length)['input_ids']
sub2word_map_src = []
for i, word_list in enumerate(token_src):
  sub2word_map_src += [i for x in word_list]
sub2word_map_tgt = []
for i, word_list in enumerate(token_tgt):
  sub2word_map_tgt += [i for x in word_list]

# alignment
align_layer = 8
threshold = 1e-3
model.eval()
with torch.no_grad():
  out_src = model(ids_src.unsqueeze(0), output_hidden_states=True)[2][align_layer][0, 1:-1]
  out_tgt = model(ids_tgt.unsqueeze(0), output_hidden_states=True)[2][align_layer][0, 1:-1]

  dot_prod = torch.matmul(out_src, out_tgt.transpose(-1, -2))

  softmax_srctgt = torch.nn.Softmax(dim=-1)(dot_prod)
  softmax_tgtsrc = torch.nn.Softmax(dim=-2)(dot_prod)

  softmax_inter = (softmax_srctgt > threshold)*(softmax_tgtsrc > threshold)

align_subwords = torch.nonzero(softmax_inter, as_tuple=False)
align_words = set()
for i, j in align_subwords:
  align_words.add( (sub2word_map_src[i], sub2word_map_tgt[j]) )

# printing
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

for i, j in sorted(align_words):
  print(f'{color.BOLD}{color.BLUE}{sent_src[i]}{color.END}==={color.BOLD}{color.RED}{sent_tgt[j]}{color.END}')

Situation===疫情
Ministry===部于
Health===卫生
and===和
Population===人口
the===刚果共和国
Republic===刚果共和国
of===刚果共和国
the===刚果共和国
Congo,===刚果共和国
declared===宣布
on===年
17===17
July===7
2023===2023
suspected===疑似
outbreak===发生
outbreak===疫情
of===的
gastroenteritis,===胃肠炎
typhoid===伤寒
typhoid===志贺氏菌
fever,===、
fever,===病
shigellosis,===志贺氏菌
and===和
cholera===霍乱
causes.Since===。
causes.Since===自
first===首例
case===首例
was===日
reported===报告
on===年
28===28
June===6
2023,===2023
as===截至
of===截至
29===29
August===8
2023,===2023
2023,===，
of===了
2389===2389
suspected===疑似病例
cases===疑似病例
reported,===报告
reported,===，
including===包括
52===52
deaths===死亡
(overall===（
(overall===总
Rate===病死率
(CFR)===为
2.2%).Laboratory===2.2%
2.2%).Laboratory===）
2.2%).Laboratory===。
analyses===分析
performed===进行
by===的
National===国家
Public===公共卫生
Health===公共卫生
Laboratory,===实验室
identified===确定
83===83
cases===例志
cases===病例
Shigella,===例志
Shigella,===、
22===22
Salmonella===伤寒
Salmonella===沙门氏菌
Salmonella===霍乱弧菌
and===和
21===21
cases===例